In [3]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

MODEL_NAME = "skt/kogpt2-base-v2"
MODEL_PATH = "./models"
SEQ_LEN = 50
TOKENS_DICT = {
    'eos_token':'</s>',
    'pad_token':'<pad>',
    'additional_special_tokens':['<context>', '<slogan>'],
}

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens(TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

print(tokenizer.special_tokens_map)
print(model.resize_token_embeddings(len(tokenizer)))

{'eos_token': '</s>', 'pad_token': '<pad>', 'additional_special_tokens': "['<context>', '<slogan>']"}
Embedding(51202, 768)


In [4]:
# HuggingFace에서 top k와 top p로 함수 샘플링
import torch
import torch.nn.functional as F
from tqdm import trange


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):

    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # top-k의 마지막 토큰보다 확률이 낮은 모든 토큰을 제거
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # 임계값 이상의 누적 확률을 가진 토큰 제거
        sorted_indices_to_remove = cumulative_probs > top_p
        
        # 첫 번째 토큰도 임계값보다 높게 유지하려면 인덱스를 오른쪽으로 이동
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # 정렬된 텐서를 원래 인덱싱에 분산
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


# HuggingFace에서 컨텍스트/슬로건 분리 작업에 맞게 조정됨
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # CTRL의 반복 페널티(https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated


In [6]:
context = "드립 커피 전문점"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * SEQ_LEN
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]


epochs = [2, 5, 10]
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+ '/' + f'processed_slogan_{epoch}epoch_model.pth'))
  model.eval()

  # 최개길이 20의 20개의 슬로건 샘플
  # 확률분포를 조금 뾰족하게 하여 확률값이 높은 토큰이 살짝 더 잘나오도록 (temperature=0.9)
  # top_k 샘플링을 적용하여 확률값이 낮은 토큰들은 후보 단어에서 배제 (top_k=5)
  generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, temperature=0.9, top_k=5, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<slogan>')[1]
    print(slogan)   

100%|██████████| 20/20 [00:14<00:00,  1.35it/s]




--- Generated Slogans ---

 당신의 아삭하게 
 당신의 아빠의 
 우리아이 한국을 더, 당신을 위한 내 순하게  
 대한민국, 내일을 위해 
 당신의 모든 모든 일상을 위한 
 우리, 당신이 행복을 위해 
 모든 일상을 위해 
 대한민국 한국을 바꾸다, 당신을 바꾸다 
 우리 가족의 당신이 더 
 대한민국, 내일의 당신은 한국을 위한 
 당신을 위한 당신이 더 더 
 당신은 행복을 위한 당상을 바꾸다 
 당신은 행복한 
 모든 일상을 바꾸다 
 당신의 모든 모든 일상을 바꾸다 
 우리만의 당신이 내 순상을 위한 
 모든 일상을 바꾸다 
 당신의 모든 일상을 위한 힘다 
 당신이 행복이까 
 당신의 한끼, 


100%|██████████| 20/20 [00:14<00:00,  1.35it/s]




--- Generated Slogans ---

 Its them them in worry Fifes 
 Im in your do 
 당신의 모든 순간, 모든 것 
 당신의 새로운 새로운 모든 순간에 
 내일의 커피의 기준 
 ILNEMEESMEME 
 스타일이 
 내 몸에 빛나는 모든 순향존RE 
 스타일을 위해 
 내일도 빛나는 순간 
 내일을 바꾸다 
 스타일리드 
 내 몸에 좋은 카트 
 ILL IMEE TUST 
 THIS TOREAINE 
 내 몸이 빛나는 세상 
 IMEMEMEATEMEIE 
 IN IUTME ISMMEININE 
 스타일이 커피를 만나다 
 스타일은 카카를 바꾸다 


100%|██████████| 20/20 [00:14<00:00,  1.34it/s]



--- Generated Slogans ---

 커피 예술을 하다 
 커피 예술을 만듭니다 
 한 번에서 커피이 다했다, 당신을 더 가까이 
 커피의 시작이 쌓인다 
 커피을 담다, 판타지 않는다 
 커피를 담다, 판타스 
 당신을 생각하는 세상의 모든 프리미엄가 
 프리미엄 커피 
 커피를 위한 모든 순간 
 커피을 담다 
 프리미엄 커피피스 
 당신의 선택의 새로운 기준 
 프리미엄 커피, 위대한 기준 
 당신의 마스터피스피스피스 
 당신이 원하는 판타지,라이 
 커피가 
 당신을 위한 퍼포미엄 혜택 
 커피의 새로운 기준 
 커피를 위한 모든 것 
 당신은 기다리던 커피의 시작 


In [7]:
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+ '/' + f'processed_slogan_{epoch}epoch_model.pth'))
  model.eval()

  # 최개길이 20의 20개의 슬로건 샘플
  # 확률분포를 조금 뾰족하게 하여 확률값이 높은 토큰이 살짝 더 잘나오도록 (temperature=0.9)
  # top_k 샘플링과 top_p 샘플링을 동시에 적용하여 확률값이 낮은 토큰들은 후보 단어에서 배제 (top_k=50, top_p=0.95)
  generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, temperature=0.9, top_k=20, top_p=0.95, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<slogan>')[1]
    print(slogan)   

100%|██████████| 20/20 [00:15<00:00,  1.32it/s]




--- Generated Slogans ---

 내 차만의 더 행복한 모든 수분의 대한민국 
 아끼, 
 당신의 한 번만, 이 알아 
 우리만의 모든 순간, 
 피부는 우리 가족부터 내 한끼 
 이 이상의 
 좋은 손만의 일상을 위한 기술 
 당신만 
 나를 위한 당신이 더 수 없는 아칠든다 
 당신의 행복을 깨우다 
 대한민국를 위한 금융 
 요즘 한분의 한국을 위한 
 우리에서도 our 
 요즘, 더, 일상에 더, 그 
 이 장일의 대한민국 
 일상을 만듭니다 
 한 잔 땐 
 대한민국 our Your lour Lree 
 피침을 더 
 모든 손안의 대한민국은 


100%|██████████| 20/20 [00:15<00:00,  1.32it/s]




--- Generated Slogans ---

 스타일은 스타일은 여좀 
 내일운 무피크워, 모든 순한 모든국에 보습 
 당신의 모든 것을 지배하는 
 Lont woring Lifortion bot stains your difers 
 내일은 여정이 필요한 
 Letter, You 
 우리아이 한 판도 유 번에 
 모든 순간을 위해 
 당신의 모든 순간에 
 스타일을 찾아보세요 
 THE TY IY OUY IOTHIF 
 당신의 모든 것은, SUKGTE NELFE EME 
 내일의 한 발만의 커피 
 한 가지거운 내 몸에 한피 
 IUT IARUTANL 
 스타일리거운 요리가 만든 
 새로운 클래스를 바꾸다 
 스타일로 선택한 모든 순간 
 올 겨울도 안 짜릿하게 
 스타일이의 새로운 차이 


100%|██████████| 20/20 [00:15<00:00,  1.32it/s]



--- Generated Slogans ---

 커피와 함께 맛있게 
 그 맛의 시작 
 그 시절 한 판타, 또 하나의 완벽 
 당신이 기다리던 그릭 친구 
 건강한 커피라이프를 만나다 
 여행도 다 가격해? 
 한 모금까지 풀코스 
 건강한 커피 
 또짠 슈퍼저몰을 시작 
 커피를 내리다 
 당신을 건강하게 준비할 때 
 나만의 슈퍼 슈퍼콘 
 당신의 새로운 차원의 프리미엄피스 
 커피라이라이트로 
 나를 위해 
 여행의 시작은 아름답다 
 오늘도 맛있게 
 커피가 바로 지금, 바로 
 오늘을 커피를 만듭니다 
 커피도 
